# Evaluation Notebook
This notebook loads a JSONL file containing entries with `query`, `gold_answer`, and `predicted_answer`, and computes evaluation metrics:
- Exact Match (EM)
- Numeric tolerance match (absolute + relative)
- Token-level F1


In [1]:
import json
import math
from typing import List, Dict


def load_jsonl(path: str) -> List[Dict]:
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def normalize(text: str) -> str:
    return text.strip().lower() if isinstance(text, str) else str(text)

def exact_match(gold, pred):
    return normalize(gold) == normalize(pred)

def numeric_match(gold, pred, abs_tol=1e-6, rel_tol=1e-3):
    try:
        g = float(gold)
        p = float(pred)
        return abs(g - p) <= abs_tol or abs(g - p) / max(abs(g), 1e-9) <= rel_tol
    except:
        return False

def token_f1(gold, pred):
    g_tokens = normalize(gold).split()
    p_tokens = normalize(pred).split()
    if len(g_tokens) == 0 or len(p_tokens) == 0:
        return 0
    overlap = len(set(g_tokens) & set(p_tokens))
    if overlap == 0:
        return 0
    precision = overlap / len(p_tokens)
    recall = overlap / len(g_tokens)
    return 2 * precision * recall / (precision + recall)

def mae_error(gold,pred):
        try:
            g = float(gold)
            p = float(pred)
            return abs(p - g)
        except:
            return None
        
def rmse_component(gold, pred):
    """
    Returns the squared error (p - g)^2 for one gold/pred pair.
    Returns None if not numeric.
    """
    try:
        g = float(gold)
        p = float(pred)
        return (p - g) ** 2
    except:
        return None

        
def numeric_distances(gold, pred):
    """
    Returns (abs_distance, relative_distance) for one gold/pred pair.
    Returns (None, None) if not numeric.
    """
    try:
        g = float(gold)
        p = float(pred)
        abs_dist = abs(p - g)
        rel_dist = abs_dist / (abs(g) + 1e-9)
        return abs_dist, rel_dist
    except:
        return None, None



In [2]:
print(numeric_match("25.0", "25.0"))  # Example usage

True


In [3]:
import math

# ---------------------------------------------------------
# 1. Basic parser: convert whitespace-separated table into columns
# ---------------------------------------------------------

def parse_table_to_columns(text, expected_cols=None):
    """
    Parse a whitespace-separated table into column lists.

    Features:
    - If expected_cols is provided, the parser will reshape rows to that width.
    - Supports optional headers (removed if detected).
    - Returns: {col_index: [values...]}
    """

    if not isinstance(text, str) or not text.strip():
        return {}

    # Tokenize
    tokens = text.split()
    rows = []
    current = []

    for tok in tokens:
        current.append(tok)
        # If expected_cols known → force row breaks
        if expected_cols and len(current) == expected_cols:
            rows.append(current)
            current = []

    # Leftover row (only if we don't have expected_cols)
    if current and not expected_cols:
        rows.append(current)

    def looks_like_header(row):
        # Header = mostly strings
        string_count = sum(1 for x in row if not is_numeric(x))
        return string_count >= len(row) * 0.7

    if rows and looks_like_header(rows[0]):
        header = rows.pop(0)  # drop it
    else:
        header = None

    if expected_cols:
        norm_rows = []
        for r in rows:
            if len(r) < expected_cols:
                r = r + [None] * (expected_cols - len(r))
            elif len(r) > expected_cols:
                r = r[:expected_cols]
            norm_rows.append(r)
        rows = norm_rows

    columns = {}
    for r in rows:
        for ci, val in enumerate(r):
            columns.setdefault(ci, []).append(val)

    return columns

# ---------------------------------------------------------
# 2. Type detection
# ---------------------------------------------------------

def is_numeric(x):
    try:
        float(x)
        return True
    except:
        return False


# ---------------------------------------------------------
# 3. Numeric distance (MAE)
# ---------------------------------------------------------

def numeric_distance(gold_vals, pred_vals):
    """
    Compute mean absolute error between shared numeric entries.
    Rows beyond the predicted table length are ignored.
    Missing predicted values → penalty via None handling.
    """
    n = min(len(gold_vals), len(pred_vals))
    if n == 0:
        return None

    errs = []
    for g, p in zip(gold_vals[:n], pred_vals[:n]):
        try:
            errs.append(abs(float(p) - float(g)))
        except:
            # Predicted non-numeric for numeric column → heavy penalty
            return None

    return sum(errs) / len(errs) if errs else None


# ---------------------------------------------------------
# 4. String distance (very simple Levenshtein-like metric)
# ---------------------------------------------------------

def string_distance(gold_vals, pred_vals):
    """
    Compute average normalized edit distance between string cells.
    Lower is better; normalized to between 0 and 1.
    Uses simple character-based Levenshtein distance.
    """

    def lev(a, b):
        # classic DP edit distance
        dp = [[i + j if i * j == 0 else 0 for j in range(len(b)+1)]
              for i in range(len(a)+1)]

        for i in range(1, len(a)+1):
            for j in range(1, len(b)+1):
                dp[i][j] = min(
                    dp[i-1][j] + 1,      # delete
                    dp[i][j-1] + 1,      # insert
                    dp[i-1][j-1] + (a[i-1] != b[j-1])  # replace
                )
        return dp[-1][-1]

    n = min(len(gold_vals), len(pred_vals))
    if n == 0:
        return None

    errs = []
    for g, p in zip(gold_vals[:n], pred_vals[:n]):
        g = str(g)
        p = str(p)
        dist = lev(g, p)
        denom = max(len(g), 1)
        errs.append(dist / denom)   # 0..1 normalized

    return sum(errs) / len(errs) if errs else None


# ---------------------------------------------------------
# 5. Simple Column-Based Evaluator
# ---------------------------------------------------------

def evaluate_columns(gold_text, pred_text, expected_cols=None):
    """
    Returns a dict:
        {
          column_index: {
             "type": "numeric" or "string",
             "score": ...,
             "coverage": ...
          },
          ...
        }

    We compute:
    - numeric MAE for numeric columns
    - string edit distance for non-numeric columns

    Coverage:
    = (# predicted rows for that column) / (# gold rows)
    """

    gold_cols = parse_table_to_columns(gold_text, expected_cols)
    pred_cols = parse_table_to_columns(pred_text, expected_cols)

    all_col_indices = set(gold_cols.keys()) | set(pred_cols.keys())
    results = {}

    for ci in all_col_indices:
        g_vals = gold_cols.get(ci, [])
        p_vals = pred_cols.get(ci, [])

        # coverage: how much of the gold column is predicted?
        coverage = min(len(g_vals), len(p_vals)) / max(len(g_vals), 1)

        # detect type: majority numeric in gold column → numeric
        numeric_count = sum(is_numeric(v) for v in g_vals)
        is_num = numeric_count > len(g_vals) / 2.0

        if is_num:
            score = numeric_distance(g_vals, p_vals)
            col_type = "numeric"
        else:
            score = string_distance(g_vals, p_vals)
            col_type = "string"

        results[ci] = {
            "type": col_type,
            "coverage": coverage,
            "score": score
        }

    return results


## Qwen Evaluation

In [4]:
data = load_jsonl("result_qwen.jsonl")

em_scores = []
numeric_scores = []
f1_scores = []

mae_list = []
rmse_list = []
abs_dist_list = []
rel_dist_list = []

column_eval_list = []

for item in data:
    gold = item.get("gold", "")
    pred = item.get("predicted", "")
    total_cols = item.get("columns", None)

    # basic metrics
    em_scores.append(exact_match(gold, pred))
    numeric_scores.append(numeric_match(gold, pred))
    f1_scores.append(token_f1(gold, pred))

    # per-item numeric metrics
    mae = mae_error(gold, pred)
    rmse = rmse_component(gold, pred)
    abs_dist, rel_dist = numeric_distances(gold, pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    abs_dist_list.append(abs_dist)
    rel_dist_list.append(rel_dist)
    try:
        column_eval = evaluate_columns(gold, pred, total_cols)
        column_eval_list.append(column_eval)
    except:
        column_eval_list.append({})


with open("qwen_eval_results.txt", "w") as f:
    for i, item in enumerate(data):
        f.write(f"Item {i+1}:\n")
        f.write(f"Gold: {item.get('gold', '')}\n")
        f.write(f"Predicted: {item.get('predicted', '')}\n")

        f.write(f"Exact Match: {'✔' if em_scores[i] else '✘'}\n")
        f.write(f"Numeric Match: {'✔' if numeric_scores[i] else '✘'}\n")
        f.write(f"Token-level F1: {f1_scores[i]:.4f}\n")

        f.write(f"MAE (abs error): {mae_list[i]}\n")
        f.write(f"RMSE component (sq error): {rmse_list[i]}\n")
        f.write(f"Abs Distance: {abs_dist_list[i]}\n")
        f.write(f"Rel Distance: {rel_dist_list[i]}\n")

        f.write("Column Evaluation:\n")
        for col_index, colinfo in column_eval_list[i].items():
            if colinfo:  # Ensure colinfo is not empty
                f.write(
                    f"  Column {col_index}: "
                    f"type={colinfo['type']}, "
                    f"coverage={colinfo['coverage']:.2f}, "
                    f"score={colinfo['score']}\n"
                )
        f.write("\n")

# --------------------
# Aggregate summary
# --------------------

# Filter out None values
valid_mae = [x for x in mae_list if x is not None]
valid_rmse = [x for x in rmse_list if x is not None]

final_mae = sum(valid_mae) / len(valid_mae) if valid_mae else None
final_rmse = math.sqrt(sum(valid_rmse) / len(valid_rmse)) if valid_rmse else None

print("✔ Exact Match Accuracy:", sum(em_scores) / len(em_scores))
print("✔ Numeric Match Accuracy:", sum(numeric_scores) / len(numeric_scores))
print("✔ Token-level F1 (avg):", sum(f1_scores) / len(f1_scores))
print("✔ MAE (avg):", final_mae)
print("✔ RMSE:", final_rmse)

✔ Exact Match Accuracy: 0.18181818181818182
✔ Numeric Match Accuracy: 0.13636363636363635
✔ Token-level F1 (avg): 0.14782236596631335
✔ MAE (avg): 18007.184825
✔ RMSE: 36014.36965


## Mistral Evaluation

In [5]:
data = load_jsonl("result_pixtral.jsonl")

em_scores = []
numeric_scores = []
f1_scores = []

mae_list = []
rmse_list = []
abs_dist_list = []
rel_dist_list = []

column_eval_list = []

for item in data:
    gold = item.get("gold", "")
    pred = item.get("predicted", "")
    total_cols = item.get("columns", None)

    # basic metrics
    em_scores.append(exact_match(gold, pred))
    numeric_scores.append(numeric_match(gold, pred))
    f1_scores.append(token_f1(gold, pred))

    # per-item numeric metrics
    mae = mae_error(gold, pred)
    rmse = rmse_component(gold, pred)
    abs_dist, rel_dist = numeric_distances(gold, pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    abs_dist_list.append(abs_dist)
    rel_dist_list.append(rel_dist)
    try:
        column_eval = evaluate_columns(gold, pred, total_cols)
        column_eval_list.append(column_eval)
    except:
        column_eval_list.append({})


with open("mistral_eval_results.txt", "w") as f:
    for i, item in enumerate(data):
        f.write(f"Item {i+1}:\n")
        f.write(f"Gold: {item.get('gold', '')}\n")
        f.write(f"Predicted: {item.get('predicted', '')}\n")

        f.write(f"Exact Match: {'✔' if em_scores[i] else '✘'}\n")
        f.write(f"Numeric Match: {'✔' if numeric_scores[i] else '✘'}\n")
        f.write(f"Token-level F1: {f1_scores[i]:.4f}\n")

        f.write(f"MAE (abs error): {mae_list[i]}\n")
        f.write(f"RMSE component (sq error): {rmse_list[i]}\n")
        f.write(f"Abs Distance: {abs_dist_list[i]}\n")
        f.write(f"Rel Distance: {rel_dist_list[i]}\n")

        f.write("Column Evaluation:\n")
        for col_index, colinfo in column_eval_list[i].items():
            if colinfo:  # Ensure colinfo is not empty
                f.write(
                    f"  Column {col_index}: "
                    f"type={colinfo['type']}, "
                    f"coverage={colinfo['coverage']:.2f}, "
                    f"score={colinfo['score']}\n"
                )
        f.write("\n")

# --------------------
# Aggregate summary
# --------------------

# Filter out None values
valid_mae = [x for x in mae_list if x is not None]
valid_rmse = [x for x in rmse_list if x is not None]

final_mae = sum(valid_mae) / len(valid_mae) if valid_mae else None
final_rmse = math.sqrt(sum(valid_rmse) / len(valid_rmse)) if valid_rmse else None

print("✔ Exact Match Accuracy:", sum(em_scores) / len(em_scores))
print("✔ Numeric Match Accuracy:", sum(numeric_scores) / len(numeric_scores))
print("✔ Token-level F1 (avg):", sum(f1_scores) / len(f1_scores))
print("✔ MAE (avg):", final_mae)
print("✔ RMSE:", final_rmse)

✔ Exact Match Accuracy: 0.2727272727272727
✔ Numeric Match Accuracy: 0.045454545454545456
✔ Token-level F1 (avg): 0.0886991667641363
✔ MAE (avg): 0.5
✔ RMSE: 0.7071067811865476


## LLama Evaluation

In [6]:
data = load_jsonl("result_llama_mav.jsonl")

em_scores = []
numeric_scores = []
f1_scores = []

mae_list = []
rmse_list = []
abs_dist_list = []
rel_dist_list = []

column_eval_list = []

for item in data:
    gold = item.get("gold", "")
    pred = item.get("predicted", "")
    total_cols = item.get("columns", None)

    # basic metrics
    em_scores.append(exact_match(gold, pred))
    numeric_scores.append(numeric_match(gold, pred))
    f1_scores.append(token_f1(gold, pred))

    # per-item numeric metrics
    mae = mae_error(gold, pred)
    rmse = rmse_component(gold, pred)
    abs_dist, rel_dist = numeric_distances(gold, pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    abs_dist_list.append(abs_dist)
    rel_dist_list.append(rel_dist)
    try:
        column_eval = evaluate_columns(gold, pred, total_cols)
        column_eval_list.append(column_eval)
    except:
        column_eval_list.append({})


with open("llama_mav_eval_results.txt", "w") as f:
    for i, item in enumerate(data):
        f.write(f"Item {i+1}:\n")
        f.write(f"Gold: {item.get('gold', '')}\n")
        f.write(f"Predicted: {item.get('predicted', '')}\n")

        f.write(f"Exact Match: {'✔' if em_scores[i] else '✘'}\n")
        f.write(f"Numeric Match: {'✔' if numeric_scores[i] else '✘'}\n")
        f.write(f"Token-level F1: {f1_scores[i]:.4f}\n")

        f.write(f"MAE (abs error): {mae_list[i]}\n")
        f.write(f"RMSE component (sq error): {rmse_list[i]}\n")
        f.write(f"Abs Distance: {abs_dist_list[i]}\n")
        f.write(f"Rel Distance: {rel_dist_list[i]}\n")

        f.write("Column Evaluation:\n")
        for col_index, colinfo in column_eval_list[i].items():
            if colinfo:  # Ensure colinfo is not empty
                f.write(
                    f"  Column {col_index}: "
                    f"type={colinfo['type']}, "
                    f"coverage={colinfo['coverage']:.2f}, "
                    f"score={colinfo['score']}\n"
                )
        f.write("\n")

# --------------------
# Aggregate summary
# --------------------

# Filter out None values
valid_mae = [x for x in mae_list if x is not None]
valid_rmse = [x for x in rmse_list if x is not None]

final_mae = sum(valid_mae) / len(valid_mae) if valid_mae else None
final_rmse = math.sqrt(sum(valid_rmse) / len(valid_rmse)) if valid_rmse else None

print("✔ Exact Match Accuracy:", sum(em_scores) / len(em_scores))
print("✔ Numeric Match Accuracy:", sum(numeric_scores) / len(numeric_scores))
print("✔ Token-level F1 (avg):", sum(f1_scores) / len(f1_scores))
print("✔ MAE (avg):", final_mae)
print("✔ RMSE:", final_rmse)

✔ Exact Match Accuracy: 0.13636363636363635
✔ Numeric Match Accuracy: 0.09090909090909091
✔ Token-level F1 (avg): 0.1566753653115568
✔ MAE (avg): 20410.172325
✔ RMSE: 40799.350054498216


## Gemini Evaluation

In [7]:
data = load_jsonl("result_gemini.jsonl")

em_scores = []
numeric_scores = []
f1_scores = []

mae_list = []
rmse_list = []
abs_dist_list = []
rel_dist_list = []

column_eval_list = []

for item in data:
    gold = item.get("gold", "")
    pred = item.get("predicted", "")
    total_cols = item.get("columns", None)

    # basic metrics
    em_scores.append(exact_match(gold, pred))
    numeric_scores.append(numeric_match(gold, pred))
    f1_scores.append(token_f1(gold, pred))

    # per-item numeric metrics
    mae = mae_error(gold, pred)
    rmse = rmse_component(gold, pred)
    abs_dist, rel_dist = numeric_distances(gold, pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    abs_dist_list.append(abs_dist)
    rel_dist_list.append(rel_dist)
    try:
        column_eval = evaluate_columns(gold, pred, total_cols)
        column_eval_list.append(column_eval)
    except:
        column_eval_list.append({})


with open("gemini_results.txt", "w") as f:
    for i, item in enumerate(data):
        f.write(f"Item {i+1}:\n")
        f.write(f"Gold: {item.get('gold', '')}\n")
        f.write(f"Predicted: {item.get('predicted', '')}\n")

        f.write(f"Exact Match: {'✔' if em_scores[i] else '✘'}\n")
        f.write(f"Numeric Match: {'✔' if numeric_scores[i] else '✘'}\n")
        f.write(f"Token-level F1: {f1_scores[i]:.4f}\n")

        f.write(f"MAE (abs error): {mae_list[i]}\n")
        f.write(f"RMSE component (sq error): {rmse_list[i]}\n")
        f.write(f"Abs Distance: {abs_dist_list[i]}\n")
        f.write(f"Rel Distance: {rel_dist_list[i]}\n")

        f.write("Column Evaluation:\n")
        for col_index, colinfo in column_eval_list[i].items():
            if colinfo:  # Ensure colinfo is not empty
                f.write(
                    f"  Column {col_index}: "
                    f"type={colinfo['type']}, "
                    f"coverage={colinfo['coverage']:.2f}, "
                    f"score={colinfo['score']}\n"
                )
        f.write("\n")

# --------------------
# Aggregate summary
# --------------------

# Filter out None values
valid_mae = [x for x in mae_list if x is not None]
valid_rmse = [x for x in rmse_list if x is not None]

final_mae = sum(valid_mae) / len(valid_mae) if valid_mae else None
final_rmse = math.sqrt(sum(valid_rmse) / len(valid_rmse)) if valid_rmse else None

print("✔ Exact Match Accuracy:", sum(em_scores) / len(em_scores))
print("✔ Numeric Match Accuracy:", sum(numeric_scores) / len(numeric_scores))
print("✔ Token-level F1 (avg):", sum(f1_scores) / len(f1_scores))
print("✔ MAE (avg):", final_mae)
print("✔ RMSE:", final_rmse)

✔ Exact Match Accuracy: 0.2727272727272727
✔ Numeric Match Accuracy: 0.13636363636363635
✔ Token-level F1 (avg): 0.11419277520972436
✔ MAE (avg): 0.0
✔ RMSE: 0.0


## DeepSeel Evaluation

In [8]:
data = load_jsonl("result_deep.jsonl")

em_scores = []
numeric_scores = []
f1_scores = []

mae_list = []
rmse_list = []
abs_dist_list = []
rel_dist_list = []

column_eval_list = []

for item in data:
    gold = item.get("gold", "")
    pred = item.get("predicted", "")
    total_cols = item.get("columns", None)

    # basic metrics
    em_scores.append(exact_match(gold, pred))
    numeric_scores.append(numeric_match(gold, pred))
    f1_scores.append(token_f1(gold, pred))

    # per-item numeric metrics
    mae = mae_error(gold, pred)
    rmse = rmse_component(gold, pred)
    abs_dist, rel_dist = numeric_distances(gold, pred)

    mae_list.append(mae)
    rmse_list.append(rmse)
    abs_dist_list.append(abs_dist)
    rel_dist_list.append(rel_dist)
    try:
        column_eval = evaluate_columns(gold, pred, total_cols)
        column_eval_list.append(column_eval)
    except:
        column_eval_list.append({})


with open("deep_results.txt", "w") as f:
    for i, item in enumerate(data):
        f.write(f"Item {i+1}:\n")
        f.write(f"Gold: {item.get('gold', '')}\n")
        f.write(f"Predicted: {item.get('predicted', '')}\n")

        f.write(f"Exact Match: {'✔' if em_scores[i] else '✘'}\n")
        f.write(f"Numeric Match: {'✔' if numeric_scores[i] else '✘'}\n")
        f.write(f"Token-level F1: {f1_scores[i]:.4f}\n")

        f.write(f"MAE (abs error): {mae_list[i]}\n")
        f.write(f"RMSE component (sq error): {rmse_list[i]}\n")
        f.write(f"Abs Distance: {abs_dist_list[i]}\n")
        f.write(f"Rel Distance: {rel_dist_list[i]}\n")

        f.write("Column Evaluation:\n")
        for col_index, colinfo in column_eval_list[i].items():
            if colinfo:  # Ensure colinfo is not empty
                f.write(
                    f"  Column {col_index}: "
                    f"type={colinfo['type']}, "
                    f"coverage={colinfo['coverage']:.2f}, "
                    f"score={colinfo['score']}\n"
                )
        f.write("\n")

# --------------------
# Aggregate summary
# --------------------

# Filter out None values
valid_mae = [x for x in mae_list if x is not None]
valid_rmse = [x for x in rmse_list if x is not None]

final_mae = sum(valid_mae) / len(valid_mae) if valid_mae else None
final_rmse = math.sqrt(sum(valid_rmse) / len(valid_rmse)) if valid_rmse else None

print("✔ Exact Match Accuracy:", sum(em_scores) / len(em_scores))
print("✔ Numeric Match Accuracy:", sum(numeric_scores) / len(numeric_scores))
print("✔ Token-level F1 (avg):", sum(f1_scores) / len(f1_scores))
print("✔ MAE (avg):", final_mae)
print("✔ RMSE:", final_rmse)

✔ Exact Match Accuracy: 0.22727272727272727
✔ Numeric Match Accuracy: 0.0
✔ Token-level F1 (avg): 0.07319857525055883
✔ MAE (avg): 1.0
✔ RMSE: 1.0
